In [ ]:
import os
import json
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import seaborn as sns
matplotlib.rcParams['mathtext.fontset'] = 'cm'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
matplotlib.rcParams["font.size"] = 30

In [ ]:
def load_data(filename):
    with open(filename, "r") as file:
        data = pd.DataFrame(json.load(file))
    data["total_dry_time"]        = data["dry_run_time"]          + data["creation_time"] + data["import_time"]
    data["total_simulation_time"] = data["simulation_times_mean"] + data["creation_time"] + data["import_time"]
    return data

In [ ]:
def plot_scaling(quantity, precision, circuit="qft"):

    # Load data
    cpu = load_data("../library_cpu.dat")
    gpu = load_data("../library_gpu.dat")
    gpu["library"] += " GPU"
    data = pd.concat([cpu, gpu])

    
    # Settings
    libraries = ["qibo", "qiskit", "qsim", "qibo GPU", "qiskit-gpu GPU", "qsim-cuquantum GPU", "qcgpu GPU"]

    # Prepare GPU data
    condition = (data["circuit"] == circuit) & (data["precision"] == precision)
    data = {k: data[(data["library"] == k) & condition] for k in libraries}

    # Plot data
    cp = sns.color_palette("colorblind", len(libraries))
    plt.figure(figsize=(16, 9))
    plt.semilogy(data["qibo"]["nqubits"], data["qibo"][quantity], marker="s", markersize=10,
                 color=cp[0], linewidth=3.0, label="qibo")
    plt.semilogy(data["qiskit"]["nqubits"], data["qiskit"][quantity], marker="o", markersize=10,
                 color=cp[1], linewidth=3.0, label="qiskit")
    plt.semilogy(data["qsim"]["nqubits"], data["qsim"][quantity], marker="^", markersize=10,
                 color=cp[2], linewidth=3.0, label="qsim")
    plt.semilogy(data["qibo GPU"]["nqubits"], data["qibo GPU"][quantity], marker="o", markersize=10,
                 color=cp[3], linewidth=3.0, label="qibo GPU")
    plt.semilogy(data["qiskit-gpu GPU"]["nqubits"], data["qiskit-gpu GPU"][quantity], marker="^", markersize=10,
                 color=cp[4], linewidth=3.0, label="qiskit GPU")
    plt.semilogy(data["qsim-cuquantum GPU"]["nqubits"], data["qsim-cuquantum GPU"][quantity],
                 color=cp[5], linewidth=3.0, label="qsim-cuquantum GPU", marker="D", markersize=10)
    plt.semilogy(data["qcgpu GPU"]["nqubits"], data["qcgpu GPU"][quantity],
                 color=cp[5], linewidth=3.0, label="qcgpu", marker="*", markersize=10)


    plt.title(f"{circuit}, {precision} precision")
    plt.xlabel("Number of qubits")
    if quantity == "total_dry_time":
        plt.ylabel("Total dry run time (sec)")
    elif quantity == "total_simulation_time":
        plt.ylabel("Total simulation time (sec)")
    plt.legend(fontsize="small", bbox_to_anchor=(1,1))
    plt.savefig(f"libraries_scaling_{quantity}_{precision}.pdf", bbox_inches="tight")

In [ ]:
plot_scaling("total_dry_time", "single")

In [ ]:
def plot_bars(precision, nqubits):

    # Load data
    cpu_data = load_data("../library_cpu.dat")
    gpu_data = load_data("../library_gpu.dat")
    gpu_data["library"] += " GPU"
    data = pd.concat([cpu_data, gpu_data])

    
    # Set plot params
    hatches = ['/', '\\', 'o', '-', 'x', '.', '*']
    width = 0.1
    quantities = ["import_time", "creation_time", "dry_run_time", "simulation_times_mean", "total_simulation_time"]
    circuit = ["qft", "variational", "supremacy", "qv", "bv"]
    libraries = ["qibo", "qiskit", "qsim", "qibo GPU", "qiskit-gpu GPU", "qsim-cuquantum GPU", "qcgpu GPU"]
    cp = sns.color_palette("pastel", len(libraries))
    ws = [-3*width, -2*width, -width, 0, width, 2 * width, 3*width]
    base_condition = (data["nqubits"] == nqubits) & (data["precision"] == precision)

    
    # Plot the results
    xvalues = np.array(range(len(circuit)))
    plt.figure(figsize=(25, 9))
    plt.title(f"{nqubits} qubits - Simulation time")
    for i, library in enumerate(libraries):
        condition = (data["library"] == library) & base_condition
        heights = {q: np.array([float(data[condition & (data["circuit"] == circ)][q]) for circ in circuit])
                   for q in quantities}

        plt.bar(xvalues + ws[i], heights["simulation_times_mean"], color=cp[i],
                align="center", width=width, alpha=1, label=library, log=True,
                edgecolor='w', hatch=hatches[i])

    plt.xticks(xvalues, circuit)
    plt.ylabel("Simulation time (sec)")
    plt.legend(fontsize="small", bbox_to_anchor=(1,1))
    plt.savefig(f"libraries_bars_{precision}_{nqubits}.pdf", bbox_inches="tight")

In [ ]:
plot_bars("single", 28)